<a href="https://colab.research.google.com/github/lanlooker/Looker-Tutorials/blob/main/03_Pause_or_Resume_Schedules_(using_requests_module).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Purpose**: This script pauses or resumes all schedules of a particular look using Looker API and Python's requests module. The same logic can be used to make changes to dashboards or lookml dashboards. 

Please click "Open in Colab" to run the script as a one-off task.  You can also download the file as a .py file to run inside a server (refer to [this page ](https://github.com/looker-open-source/sdk-codegen/blob/main/README.md#configuring-lookerini-or-env
) to learn about the init file) 

The script is out of scope for Looker chat support. For questions and comments, please post in Looker Community. 

**How to run as a one-off task in Google Colab**: 
1. Edit your client ID and secret key in the first cell 
2. Run the second cell to load functions 
3. Update the parameter in the third cell to call the functions


In [ ]:
instance = 'https://yourcompany.looker.com:19999' 

credential = {
  'client_id': 'foo',
  'client_secret': 'bar'    
}

In [ ]:
import requests
import json

def login(instance, credential):

   """ Return an access token to be used in subsequent API calls 
   An access token is only valid for 60 minutes and can not be extended"""

   instance = instance 
   credential = credential
   access_token = requests.post(instance+'/login', data=credential).json() 
   headers = {'Authorization': 'token ' +  access_token['access_token']}
   return headers


def update_schedule(look_id = 321, enabled = True):

  """ Pause a schedule plan (enabled = False) or resume a schedule plan (enabled = True) 

  Args: 
    look_id (int): id of a look (https://company.looker.com/boards/id)
    enabled (boolean): set to "True" to resume schedule, and "False" to pause schedule
  
  Returns: "Successfully updated all schedules for look id " (str)
  
  Notes: Schedules with "enabled = False" will disappear from Admin > Schedules but its data 
  can be retrived in System Activity. Once schedules are resumed with "enabled = True", they 
  will appear in Admin > Schedules 
  """
   
  headers = login(instance = instance, credential = credential)

  """ Change to <schedule_plans/dashboard/> or <schedule_plans/lookml_dashboard> 
  to retrieve schedule_plan_id for these two contents """
  get_url = instance + '/api/4.0/scheduled_plans/look/' + str(look_id)
  schedule_plans = requests.get(get_url,headers=headers).json()


  """ Execute through loops for all schedule plans belonging to one Looker type (look or dashboards)"""
  for i in range(0, len(schedule_plans)):
    schedule_plan = schedule_plans[i]
    schedule_plan['enabled'] = enabled
    data = json.dumps(schedule_plan) 
    update_url = instance + '/scheduled_plans/' + str(schedule_plan['id'])
    requests.patch(update_url, headers = headers, data = data)
    
  return "Successfully updated all schedules for look id " + str(look_id)

In [ ]:
# Put in look_id and change the enabled parameter 
update_schedule(look_id = 321, enabled = False)